# Visualizing spatial data in Python
We have a number of options for displaying spatial data using Python. Here we use folium, geopandas, and the ArcGIS Python API to display point and polygon data. 

## Getting data
First we'll fetch some NWIS gage location data to plot. 

In [ ]:
#Import pandas
import pandas as pd

In [ ]:
#Get the list of site names for NC using the NWIS API
theURL = ('https://waterdata.usgs.gov/nwis/inventory?' + \
          'state_cd=nc&' + \
          'group_key=NONE&' + \
          'format=sitefile_output&' + \
          'sitefile_output_format=rdb&' + \
          'column_name=site_no&' + \
          'column_name=station_nm&' + \
          'column_name=site_tp_cd&' + \
          'column_name=dec_lat_va&' + \
          'column_name=dec_long_va&' + \
          'column_name=drain_area_va&' + \
          'list_of_search_criteria=state_cd')
colnames=['site_no','station_nm','site_tp_cd','lat','lng','agent','datum','d_area']

#Pull the data from the URL
dfNWIS = pd.read_csv(theURL,skiprows=29,sep='\t',names=colnames,index_col='site_no')

#Drop rows with null values
dfNWIS.dropna(inplace=True)

#Display
dfNWIS.head()

## Setting up to visualize our data
We are going to construct a map, and when we do, we'll have to specify where our map should be centered and how far zoomed it should be. To compute the center, we can easily compute the median Lat and Long values from our data...

In [ ]:
#Determine the median lat/lng
medianLat = dfNWIS['lat'].median()
medianLng = dfNWIS['lng'].median()
print (medianLat,medianLng)

## Visualizing with the `folium` package
https://python-visualization.github.io/folium/

Folium is a Python wrapper for the JavaScript "Leaflet" Package...

In [ ]:
import folium
folium.__version__

In [ ]:
#Construct the map
m = folium.Map(location=[medianLat,medianLng],
               zoom_start = 7,
               tiles='Stamen Watercolor'              
             )
#Display the map
m

Try:
* Change the zoom value: does higher or lower values zoom in to a smaller area?
* Change the tiles to `Stamen Terrain`,`Stamen Watercolor`, `Stamen Toner`. [more](https://python-visualization.github.io/folium/quickstart.html#Getting-Started).

In [ ]:
#Create the marker, we'll use a circle Marker
myMarker = folium.CircleMarker(location=[medianLat,medianLng],
                               color='red',
                               fill=True,
                               fill_opacity=0.5,
                               radius=30,
                               tooltip='Map Center'
                              )
myMarker.add_to(m)
m

In [ ]:
#Recreate the map object to clear markers
m = folium.Map(location=[medianLat,medianLng],
               zoom_start = 7,
               tiles='OpenStreetMap'              
             )

#Loop through all features and add them to the map as markers
for row in dfNWIS.itertuples():
    #Get info for the record
    lat = row.lat
    lng = row.lng
    site_no = row.station_nm
    #Create the marker object, adding them to the map object
    folium.CircleMarker(location=[lat,lng],
                        color='blue',
                        fill=True,
                        fill_opacity=0.6,
                        radius=3,
                        tooltip=site_no
                       ).add_to(m)
#Show the map
m

In [ ]:
#Import the folium MarkerCluster object class
from folium.plugins import MarkerCluster

#Recreate the map object to clear markers
m = folium.Map(location=[medianLat,medianLng],
               zoom_start = 7,
               tiles='cartodbpositron'              
             )

#Create a marker cluster object
mc = MarkerCluster()

#Loop through all features and add them to the map as markers
for row in dfNWIS.itertuples():
    #Get info for the record
    lat = row.lat
    lng = row.lng
    site_no = row.station_nm
    #Create the marker object, adding them to the map object
    marker = folium.CircleMarker(location=[lat,lng],
                                 color='blue',
                                 fill=True,
                                 fill_opacity=0.6,
                                 radius=3,
                                 tooltip=site_no)
    #Add the marker to the markerCluster
    mc.add_child(marker)
    
#Add marker clusters to the map
m.add_child(mc)

#Show the map
m

---
## GeoPandas
GeoPandas has some plotting capabilities to visualize geodataframes. We can also plot geodataframes on a leaflet map by converting the dataframe to GeoJSON object. 

Resource: http://geopandas.org/mapping.html#

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
#Create point objects from our coordinate fields
thePoints = [Point(xy) for xy in zip(dfNWIS['lng'],dfNWIS['lat'])]

In [ ]:
#Convert the dataframe to a geodataframe
gdfNWIS = gpd.GeoDataFrame(dfNWIS,geometry=thePoints,crs={'init:''epsg:4326'})

In [ ]:
#Plot the data
%matplotlib inline
gdfNWIS.plot();

By importing Matplotlib's pyplot interface, we can add more mapping functionality. Specifically, once we create a figure, we can access it as the pyploy or "`plt`" object, and then we can apply aesthetics to this object.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Plot the data, this time coloring features by it's longitude value
gdfNWIS.plot('lng',          #The column with values to color 
             cmap='Reds',    #The colormap to use for the colors
             figsize=(12,8)) #The size of the map

#Add aesthetics via the matplotlib pyplt interface
plt.title("Map") #Add a title to the figure
plt.grid()       #Add gridlines
plt.plot();      #Display the plots

#### Polygon data

In [ ]:
#Read in a polygon shapefile
gdfHUCs = gpd.read_file('./data/12Digit_HUC_Subwatersheds.shp')
gdfHUCs.head(2)

In [ ]:
#Simple plot
gdfHUCs.plot();

In [ ]:
#Plot, colored by a discrete attribute
gdfHUCs.plot('HUC_8',cmap='tab20');

In [ ]:
#Dissolve on HUC_8
gdfHUC8 = gdfHUCs.dissolve(by='HUC_8')

In [ ]:
#Fix issue with pyproj 
import sys, os
pythonPath = sys.executable
pythonFolder = os.path.dirname(pythonPath)
shareFolder = os.path.join(pythonFolder,'Library','share')
os.environ["PROJ_LIB"] = shareFolder

In [ ]:
#Project and compute areas
gdfHUC8_utm = gdfHUC8.to_crs({'init':'epsg:26917'})
gdfHUC8_utm['area_m2'] = gdfHUC8_utm['geometry'].area

In [ ]:
#Plot, colored by a contiuous attribute
gdfHUC8_utm.plot('area_m2',cmap='autumn',legend=True,figsize=(15,5))
plt.grid(True)
plt.xlabel('UTM Easting')
plt.ylabel('UTM Northing');

In [ ]:
#Subset HUC 8 = 03020201 (Upper Neuse)
gdfNeuse = gdfHUCs.query('HUC_8 == "03020201"')
gdfNeuse.plot();

#### Plotting multiple layers
By assigning the first plot to a variable name (here `theMap`), we can add more layers by specifying which axis to plot the layer on top of.  

In [ ]:
#Plot the HUC8s, assigning the plot to the variable "theMap"
theMap = gdfHUC8_utm.plot('area_m2',cmap='autumn',legend=True,figsize=(15,5))
plt.grid(True)
plt.xlabel('UTM Easting')
plt.ylabel('UTM Northing')

#Plot the Neuse (projected to UTM) on top by specifing that it use the same axes
gdfNeuse.to_crs({'init':'epsg:26917'}).plot(ax=theMap,color='green');

#### Displaying a geopandas dataframe in a folium map

In [ ]:
#Create a folium map
m = folium.Map(location=[medianLat,medianLng],
               zoom_start = 8,
               tiles='OpenStreetMap'              
             )
#Convert the geopandas dataframe to a GeoJSON object
gdfNeuse_json = gdfNeuse.to_json()
#Convert the GeoJSon object to a folium layer
gdfNeuse_layer = folium.GeoJson(gdfNeuse_json)
#Add the layer to the map
gdfNeuse_layer.add_to(m)
#Show the map
m